# FDN Reverberator - DAAP Project

Students: 

*   Federico Maver ( ID: 10565151, MATR: 952840 )
*   Filippo Gualtieri ( ID: 10557065, MATR: 953546 )

In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import librosa, librosa.display
import scipy as sp
import IPython.display as ipd

In [ ]:
ipd.Image(filename = "images/FDNScheme.png")

## Params

In [ ]:
fs = 22050
N = 4
g = np.ones(N) # cambia come vuoi

t60 = 1 # reverberation time
# TODO: reverberation time should be function of frequency! from 0 to fs/2
K = 7
T=127
t60 = np.repeat(1.5, K) #! constant, need to create a graphic 'eq' to select t60
t60 = [1, 0.9, 1, 0.5, 0.4, 0.2, 0.1]
freqs = np.array([0, fs/(2*6), 2*fs/(2*6), 3*fs/(2*6), 4*fs/(2*6), 5*fs/(2*6), fs/2])
print(t60) 

# Min delay lenght
# TODO: create slider to select length (starting from min)
M_min = int(np.ceil(0.15*np.max(t60)*fs))
print(M_min)

In [ ]:
primeN = []

for num in range(2,101):
    prime = True
    for i in range(2,num):
        if (num%i==0):
            prime = False
    if prime:
       primeN.append(num)

M = []
for i in primeN:
    m = np.floor(0.5+np.log(M_min)/np.log(i))
    if i**m>M_min:
        M.append(int(i**m))

M = np.sort(M)

M = M[:N]

print(M)
# TODO: select numbers according to min desired length

## Lossless Feedback Matrix

Hadamard Matrix

In [ ]:
if((N & (N-1) == 0) and N != 0):
    Had = 1/np.sqrt(N)*sp.linalg.hadamard(N)
    print(Had)


Householder Matrix

In [ ]:
u = np.ones(N)
Hous = np.identity(N)-2/N*u*u.T
print(Hous)

In [8]:
# Case 3: Multiband filter design (preferred)
def multibandEqualizer(M_i, T, t60, freqs, fs):
    #for M_i in M:

        G_i = []
        # evaluate correct gains (one per band)
        for t60_f in t60:
            G_i.append(10**((-3*M_i)/(t60_f*fs))) 

        
        #print(G_i)

        taps = sp.signal.firwin2(numtaps=T, freq=freqs, gain=G_i, fs=fs)

        # w, h = sp.signal.freqz(taps, fs=fs)

        #plotFilter(w, h, M_i)

        return taps

In [ ]:
# #Circuit
# #Parameter initialization
# #U = [1, 0, 0, 0]

# U=np.append(np.sin(2*np.pi*440*np.arange(0,0.1,1/fs)),np.zeros(15000)) #sp.signal.unit_impulse(22050)
# B = [1,1,1,1]
# C = [1,1,1,1]
# G = [1,1,1,1]
# A = Hous
# '''                                          1 2 3 4
# #Delay lines data structure initialization   _ _ _ _  list of vectors
#                                              | | | |
#                                              | | | |
# '''
# delay_lines=[]

# for i in range(N):
#     delay_lines.append(np.zeros(M[i]))

# Output=[]
# for u in U: # For every sample of the input
#     delay_outputs = [] #Clear the list of delay outputs (N,)
#     x = []
#     for i in range(N): # For every circuit calculate output of delay lines and shift
#         delay_outputs.append(delay_lines[i][-1])
#         delay_lines[i][1:M[i]] = delay_lines[i][0:M[i]-1]
#         taps = multibandEqualizer(M[i], T, freqs, fs)
#         x.append(np.sum(np.multiply(taps, delay_lines[i][-T:])))
        

#     delay_outputs_array = np.asanyarray(x)
#     delay_output_mixed = np.dot(A,delay_outputs_array) # calculate feedback with the selected mixing matrix

#     Output.append(np.sum(np.multiply(delay_outputs_array, C)))# Calculate Output to E(z)

#     for i in range(N):
#         delay_lines[i][0] = B[i]*u + G[i] * delay_output_mixed[i] # insert the calculated input+feedback value into the first position of each delay line

# plt.figure()
# plt.plot(U)
# plt.plot(Output)

# print("Original")
# ipd.Audio(U, rate=fs)

In [ ]:
delay_lines[1][-T:].shape

In [ ]:
print("Filtered")
ipd.Audio(Output, rate=fs)



Once we have delayed the splitted signal, apply a filter (to each delay line). This filter is used to determine the $t_{60}$ of the reverb in each frequency band.

In [ ]:
def plotFilter(w, h, M_i):
        fig, ax1 = plt.subplots()
        ax1.set_title('Filter applied to delay line with M={}'.format(M_i))
        ax1.plot(w, 20 * np.log10(abs(h)), 'b')
        ax1.set_ylabel('Amplitude [dB]', color='b')
        ax1.set_xlabel('Frequency [rad/sample]')
        # ax2 = ax1.twinx()
        # angles = np.unwrap(np.angle(h))
        # ax2.plot(w, angles, 'g')
        # ax2.set_ylabel('Angle (radians)', color='g')
        # ax2.grid()
        # ax2.axis('tight')
        plt.show()

In [ ]:
# Case 1: Simple 1st order filter

def filter1st(x, M, t60, fs):
    for M_i in M:
        R_M = []
        # R_M = 1-(6.91*M_i/t60*fs)
        for t60_f in t60:
            R_M.append(10**((-3*M_i)/(t60_f*fs))) 
            
        print(R_M)

        # pole
        p_i = (R_M[0]-R_M[-1])/(R_M[0]+R_M[-1])

        # gain
        g_i = 2*(R_M[0]*R_M[-1])/(R_M[0]+R_M[-1])

        print(g_i, p_i)

        b = [0, g_i]
        a = [-p_i, 1]

        w, h = sp.signal.freqz(b,a, fs=fs)

        plotFilter(w, h, M_i)


In [ ]:
# Case 2: Orthogonalized 1st order delay filter
def ortFilter1st(x, M, t60, fs):
    for M_i in M:

        # gain
        g_i = 10**((-3*M_i)/(t60[0]*fs))

        # pole and zero
        p_i = np.log(10)/4 * np.log10(g_i) * (1-(1/(t60[-1]/t60[0])**2))

        print(g_i, p_i)

        b = [0, g_i*(1-p_i)]
        a = [-p_i, 1]

        w, h = sp.signal.freqz(b,a, fs=fs)

        plotFilter(w, h, M_i)


Spectral coloration equalizer

In [ ]:
# coefficient of the filter:
def spectralEqualizer(x, t60, fs):
    b_coeff = (1-(t60[-1]/t60[0]))/(1+(t60[-1]/t60[0]))

    b = [-b_coeff, 1]
    a = [0, 1-b_coeff]

    w, h = sp.signal.freqz(b,a, fs=fs)

    fig, ax1 = plt.subplots()
    ax1.set_title('Spectral Coloration Equalizer')
    ax1.plot(w, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Amplitude [dB]', color='b')
    ax1.set_xlabel('Frequency [rad/sample]')
    # ax2 = ax1.twinx()
    # angles = np.unwrap(np.angle(h))
    # ax2.plot(w, angles, 'g')
    # ax2.set_ylabel('Angle (radians)', color='g')
    # ax2.grid()
    # ax2.axis('tight')
    plt.show()

# Main:

In [9]:
# test signal

fs = 22050
signal = np.append(np.sin(2*np.pi*440*np.arange(0,0.1,1/fs)),np.zeros(15000)) #sp.signal.unit_impulse(22050)

In [10]:
# parameters


N = 4 # number of parallel delay lines

# weight coefficients (default to 1)
g = np.ones(N)
b = np.ones(N)
c = np.ones(N)

# reverberation time should be function of frequency, from 0 to fs/2
T=9 # filter taps
t60 = [1, 0.9, 1, 0.5, 0.4, 0.2, 0.1]
freqs = np.array([0, fs/(2*6), 2*fs/(2*6), 3*fs/(2*6), 4*fs/(2*6), 5*fs/(2*6), fs/2])
print('Frequency bands {0} & relative reverberation times {1}'.format(freqs, t60)) 

# Delay lenghts
# TODO: create slider to select length (starting from min)
M_min = int(np.ceil(0.15*np.max(t60)*fs))

primeN = []

# generate some prime numbers...
for num in range(2,101):
    prime = True
    for i in range(2,num):
        if (num%i==0):
            prime = False
    if prime:
       primeN.append(num)

# compute M starting from (different) prime numbers
M = []
for i in primeN:
    m = np.floor(0.5+np.log(M_min)/np.log(i))
    if i**m>M_min:
        M.append(int(i**m))

M = np.sort(M)

M = M[:N]

print('Selected delay lengths:',M)
# TODO: select numbers according to min desired length


Frequency bands [    0.   1837.5  3675.   5512.5  7350.   9187.5 11025. ] & relative reverberation times [1, 0.9, 1, 0.5, 0.4, 0.2, 0.1]
Selected delay lengths: [3481 3721 4096 4489]


In [11]:
# feedback matrices

# hadamard
if((N & (N-1) == 0) and N != 0):
    Had = 1/np.sqrt(N)*sp.linalg.hadamard(N)

#householder
u = np.ones(N)
Hous = np.identity(N)-2/N*u*u.T

In [17]:
# delay implementation with circular buffer
# ! contrario, va fatto per ogni delay line in //


signal = np.append(signal, np.zeros(5*np.max(M)))
out = np.zeros((N, np.max(M)+signal.size))
fb = 0.2

bufferSize = np.max(M) + 128

for i in range(M.size):

    taps = multibandEqualizer(M[i], T, t60, freqs, fs)

    y = []
    output = []

    writeIndex = M[i]
    readIndex = bufferSize-M[i]
    filterIndex = readIndex
    buffer = np.zeros(bufferSize)
    

    for sample in signal:

        y = buffer[readIndex]
        output = np.append(output, y)

        # filter
        filterIndex = readIndex
        filtered = y
        for tap in taps:

            filtered += buffer[filterIndex]*tap
            filterIndex = filterIndex -1
            if filterIndex<0:
                filterIndex = bufferSize-1
            

        x = sample + fb*filtered

        buffer[writeIndex] = x

        readIndex = readIndex + 1
        if readIndex>=bufferSize:
            readIndex = 0

        writeIndex = writeIndex + 1
        if writeIndex>=bufferSize:
            writeIndex = 0

        
    
    out[i][:output.size] = output
    


# plt.plot(out[:])
# plt.plot(signal)



# initialize: input signal u, feedback = 0

# feedback loop (for each sample):
    # here we have x as sum of input and feedback

    # apply the delays to x

    # apply the filters to x

    # take the result and apply the feedback matrix, loop back

    # take x and write the result into a buffer 

# take the output from the buffer and apply the equalization filter


# blend the result with original signal

In [ ]:
p = np.arange(10)
print(p)
print(p[6:-4:-1])